In [7]:
import pandas as pd
# 데이터 준비
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# 데이터 탐색
print(train.shape, test.shape)
train.info()
train.head()

train.isnull().sum()
train.describe()
train.describe(include='O')

# 결측치 처리 - 최소값(수치형), 최빈값(문자형)
print(train.isnull().sum(), test.isnull().sum())
train['Item_Weight'] = train['Item_Weight'].fillna(train['Item_Weight'].min())
train['Outlet_Size'] = train['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0])
test['Item_Weight'] = test['Item_Weight'].fillna(test['Item_Weight'].min())
test['Outlet_Size'] = test['Outlet_Size'].fillna(test['Outlet_Size'].mode()[0])
print(train.isnull().sum(), test.isnull().sum())

# item id 칼럼 삭제
train.drop('Item_Identifier', axis=1, inplace=True)
test.drop('Item_Identifier', axis=1, inplace=True)
print(train.shape, test.shape)

# 변수 선택(y_train -> target)
target = train.pop('Item_Outlet_Sales')

cols = ['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size',
        'Outlet_Location_Type', 'Outlet_Type']

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cols:
  train[col] = le.fit_transform(train[col])
  test[col] = le.fit_transform(test[col])

train.info()

# 검증 데이터 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestRegressor
# 모델 생성 및 학습
model = RandomForestRegressor(random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)

# 성능 평가 - 평균 제곱 오차, 제곱근 계산 함수 - np.sqrt()
from sklearn.metrics import mean_squared_error
import numpy as np

# 평균 제곱근 오차 함수
def rmse(y_test, pred):
  return mean_squared_error(y_test, pred) ** 0.5

# 평균 제곱 오차
mse = mean_squared_error(y_test, pred)
print('MSE:', mse)

# 평균 제곱근 오차
# rmse = np.sqrt(mse)
rmse = rmse(y_test, pred)
print('RMSE:', rmse)

# 테스트 데이터 측정
pred2 = model.predict(test)
pred2

# 파일 생성 및 제출
submit = pd.DataFrame({'pred': pred2})
submit.to_csv('result.csv', index=False)

pd.read_csv('result.csv')

(6818, 12) (1705, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6818 entries, 0 to 6817
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            6818 non-null   object 
 1   Item_Weight                5656 non-null   float64
 2   Item_Fat_Content           6818 non-null   object 
 3   Item_Visibility            6818 non-null   float64
 4   Item_Type                  6818 non-null   object 
 5   Item_MRP                   6818 non-null   float64
 6   Outlet_Identifier          6818 non-null   object 
 7   Outlet_Establishment_Year  6818 non-null   int64  
 8   Outlet_Size                4878 non-null   object 
 9   Outlet_Location_Type       6818 non-null   object 
 10  Outlet_Type                6818 non-null   object 
 11  Item_Outlet_Sales          6818 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 639.3+ KB
Item_Identifier        

,pred
0,1483.395742
1,905.527948
2,2067.182498
3,1667.662550
4,2609.050486
...,...
1700,298.791066
1701,800.984032
1702,3219.382688
1703,941.121616
